In [1]:
!pip install twilio python-dotenv scikit-learn pandas nltk

In [2]:
import os
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import re
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse




In [3]:
!pip install pyngrok --upgrade
from pyngrok import ngrok

In [4]:
faq = pd.read_csv("newfaq.csv")
faq

,Question,Answer
0,What are your business hours?,"We are open Monday to Friday, 9 AM – 6 PM, and..."
1,Do you deliver products?,"Yes, we deliver across Nigeria within 2–3 busi..."
2,What payment methods do you accept?,"We accept debit/credit cards, bank transfer, a..."
3,Do you have a refund policy?,"Yes, refunds are processed within 7 business d..."
4,Can I track my order?,"Yes, you will receive a tracking ID via email ..."
5,Do you offer discounts?,"Yes, we offer seasonal discounts and promotion..."
6,How can I contact customer support?,You can reach us via email at support@mybusine...
7,Do you sell internationally?,"Currently, we only ship within Nigeria but pla..."
8,Can I cancel my order?,"Yes, orders can be canceled within 24 hours of..."
9,Where are you located?,"Our head office is located at 15 Allen Avenue,..."


In [5]:
faq = pd.read_csv("newfaq.csv")
faq.columns = faq.columns.str.lower()
faq.head()


,question,answer
0,What are your business hours?,"We are open Monday to Friday, 9 AM – 6 PM, and..."
1,Do you deliver products?,"Yes, we deliver across Nigeria within 2–3 busi..."
2,What payment methods do you accept?,"We accept debit/credit cards, bank transfer, a..."
3,Do you have a refund policy?,"Yes, refunds are processed within 7 business d..."
4,Can I track my order?,"Yes, you will receive a tracking ID via email ..."


In [6]:
#train on all faq

model = make_pipeline(TfidfVectorizer(), MultinomialNB())

model.fit(faq['question'], faq['answer'])

print('Model Training complete')

Model Training complete


In [7]:
faq = pd.read_csv("newfaq.csv")
faq

,Question,Answer
0,What are your business hours?,"We are open Monday to Friday, 9 AM – 6 PM, and..."
1,Do you deliver products?,"Yes, we deliver across Nigeria within 2–3 busi..."
2,What payment methods do you accept?,"We accept debit/credit cards, bank transfer, a..."
3,Do you have a refund policy?,"Yes, refunds are processed within 7 business d..."
4,Can I track my order?,"Yes, you will receive a tracking ID via email ..."
5,Do you offer discounts?,"Yes, we offer seasonal discounts and promotion..."
6,How can I contact customer support?,You can reach us via email at support@mybusine...
7,Do you sell internationally?,"Currently, we only ship within Nigeria but pla..."
8,Can I cancel my order?,"Yes, orders can be canceled within 24 hours of..."
9,Where are you located?,"Our head office is located at 15 Allen Avenue,..."


In [8]:
faq.columns = faq.columns.str.lower()
faq.head()

,question,answer
0,What are your business hours?,"We are open Monday to Friday, 9 AM – 6 PM, and..."
1,Do you deliver products?,"Yes, we deliver across Nigeria within 2–3 busi..."
2,What payment methods do you accept?,"We accept debit/credit cards, bank transfer, a..."
3,Do you have a refund policy?,"Yes, refunds are processed within 7 business d..."
4,Can I track my order?,"Yes, you will receive a tracking ID via email ..."


In [ ]:


vectorizer = TfidfVectorizer()
faq_vectors = vectorizer.fit_transform(faq['question'])

def get_bot_response(user_query):
    user_query = user_query.lower().strip()
    user_vector = vectorizer.transform([user_query])
    similarities = cosine_similarity(user_vector, faq_vectors).flatten()
    max_sim = similarities.max()
    best_match_idx = similarities.argmax()
    
    if max_sim < 0.4:
        return ("Not sure about that, I will redirect your request to the provider. Hold on, Thank you !"
                "Kindly hold on while we connect you Thank you!")
    else:
        answer = faq.iloc[best_match_idx]['answer']
        
        return f"Sure! {answer}"
        
        
        
    
def start_chat():
    print("Chatbot:Hello! how can we help you today?\n")
    
    while True:
        user_query = input("You: ")
        
        if user_query.lower() in ['exit', 'bye', 'quit']:
            print("Chatbot: Thank you for checking our services. Have a great day!")
            break
            
        response = get_bot_response(user_query)
        print(f"Chatbot: {response}\n")
        print("Chatbot: What else can we help you with?\n")
        
start_chat()


Chatbot:Hello! how can we help you today?

You: Can i cancel my order?
Chatbot: Sure! Yes, orders can be canceled within 24 hours of purchase.

Chatbot: What else can we help you with?

You: what payments methods do you accept?
Chatbot: Sure! We accept debit/credit cards, bank transfer, and cash on delivery.

Chatbot: What else can we help you with?

You: what are your business working ours ?
Chatbot: Sure! We are open Monday to Friday, 9 AM – 6 PM, and Saturday 10 AM – 4 PM.

Chatbot: What else can we help you with?

You: do you have a refund policy?
Chatbot: Sure! Yes, refunds are processed within 7 business days of product return.

Chatbot: What else can we help you with?

You: do you deliver products ?
Chatbot: Sure! Yes, we deliver across Nigeria within 2–3 business days.

Chatbot: What else can we help you with?

You: where are you located?
Chatbot: Sure! Our head office is located at 15 Allen Avenue, Ikeja, Lagos.

Chatbot: What else can we help you with?

You: do you sell inter